In [8]:
import os
import geopandas as gpd
from owslib.wfs import WebFeatureService

wfs_url = "http://sit.comune.bolzano.it/geoserver/wfs?service=WFS"
wfs = WebFeatureService(url=wfs_url, version='2.0.0')
layers = list(wfs.contents)

incidenti = [layer for layer in layers if 'ciden' in layer]

for incidente in incidenti:   
    response = wfs.getfeature(typename=incidente)
    data = gpd.read_file(response)
    data.crs="epsg:25832"
    data = data.to_crs(epsg=4326)
    data['DTINCID']=data['DTINCID'].astype(str)
    data.DTINCID =data.DTINCID.apply(lambda x: x.replace(' 00:00:00+00:00',""))
    data['lon'] = data.geometry.x
    data['lat'] = data.geometry.y
    name = incidente.replace("Cartografia:view_","")

    # Save to GeoJSON
    geojson_path = "/Users/leonardo/Desktop/Tesi/LTSBikePlan/data/" + os.sep + name + ".geojson"
    data.to_file(geojson_path, driver="GeoJSON")

    # Save to CSV
    csv_path = "/Users/leonardo/Desktop/Tesi/LTSBikePlan/data/" + os.sep + name + ".csv"
    del data['geometry']
    data.to_csv(csv_path, index=False)

